#LoadData.ipynb: 

Loads Facemorph dataset, creating 3D NumPy arrays
called static_trials for the static files and dyn_trials for the dynamic files.
The trials arrays are of the shape (samples, rows, cols)

Can be changed to load fear_trials and anger_trials for the fear and 
anger stimuli respectively by changing the parameters to the get_files function. (Or you can uncomment the comment of code below).

For static_trials and dyn_trials, there are static_labels and dyn_labels
which label each of the trials as fear or anger. 
0 is labeled for fear, and 1 for anger.
There are also static_groups and dyn_groups 
to show which trials belong to which of the 24 participants in the dataset. 

The same thing exists for fear_trials and anger_trials, except they
are called fear_labels and anger_labels. 
0 is labeled for dynamic files, and 1 for static files.
fear_groups and anger_groups are for the groups.

This notebook is then run inside of another notebook to load the files of the dataset.
This can be accomplished by switching to the directory containing this notebook, and running it.

Example:

%cd dir_with_LoadData

%run LoadData.ipynb

Inside of this notebook is where you want to switch to the directory with the dataset files so they can be accessed.

Example:

%cd dir_with_dataset_files

Once that is setup the code in the rest of this notebook can be run to load the files.

Files in the dataset are of the form FM(V)_XY_eeg.dat where:

(V) = number of participant

X = 1 for fear, 2 for anger

Y = 1 for dynamic, 2 for static

**Switch directory and import modules**

In [ ]:
#switch to directory with dataset files
%cd single_trial_files

In [ ]:
import numpy as np
import pandas as pd
import glob
import os
from os.path import splitext
import csv

**Get data files**

In [ ]:
def get_files(fileType):
  '''
  Returns dictionary containing file names, labels, and groups

  fileType: String that determines what type of files to get.
    (Ex: 'static' grabs all of the static files and labels them as fear or anger)
            If fear or anger, then they are labeled as static or dynamic.

  '''

  files_array = np.array([])
  file_labels = np.array([], dtype=int)
  file_groups = np.array([], dtype=int)
  #fill arrays accordingly to file type
  if fileType == 'static':
    for file in glob.iglob("*2_eeg.dat"):
      files_array = np.append(files_array, file)
      file_groups = np.append(file_groups, file[2:4])
      #label 0 for fear, 1 for anger
      if file[5] == '1':
        file_labels = np.append(file_labels, 0)
      else:
        file_labels = np.append(file_labels, 1) 
  elif fileType == 'dynamic':
    for file in glob.iglob('*1_eeg.dat'):
      files_array = np.append(files_array, file)
      file_groups = np.append(file_groups, file[2:4])
      #label 0 for fear, 1 for anger
      if file[5] == '1':
        file_labels = np.append(file_labels, 0)
      else:
        file_labels = np.append(file_labels, 1) 
  elif fileType == 'fear':
    for file in glob.iglob("*_1*_eeg.dat"):
      files_array = np.append(files_array, file)
      file_groups = np.append(file_groups, file[2:4])
      #label 0 for dynamic, 1 for static
      if file[6] == '1':
        file_labels = np.append(file_labels, 0)
      else:
        file_labels = np.append(file_labels, 1) 
  else:
    for file in glob.iglob('*_2*_eeg.dat'):
      files_array = np.append(files_array, file)
      file_groups = np.append(file_groups, file[2:4])
      #label 0 for dynamic, 1 for static
      if file[6] == '1':
        file_labels = np.append(file_labels, 0)
      else:
        file_labels = np.append(file_labels, 1)

  #return dict of parameters
  d = dict()
  d['files_array'] = files_array
  d['file_groups'] = file_groups
  d['file_labels'] = file_labels
  return d

In [ ]:
#get static and dynamic files
static_dict = get_files("static")
static_files = static_dict['files_array']
static_groups = static_dict['file_groups']
static_labels = static_dict['file_labels']

dyn_dict = get_files("dynamic")
dyn_files = dyn_dict['files_array']
dyn_groups = dyn_dict['file_groups']
dyn_labels = dyn_dict['file_labels']

In [ ]:
'''
Would recommend only running this or the previous cell depending on what experiment you want to run.
They get the same files, just with different labels. It will also help save RAM in your session.

#get fear and anger files
fear_dict = get_files("fear")
fear_files = fear_dict['files_array']
fear_groups = fear_dict['file_groups']
fear_labels = fear_dict['file_labels']

anger_dict = get_files("anger")
anger_files = anger_dict['files_array']
anger_groups = anger_dict['file_groups']
anger_labels = anger_dict['file_labels']
'''

**Read files**

In [ ]:
def read_file(filename):
  '''
    Reads a file and returns 3D NumPy array of form (trials, rows, cols) 

    filename: Name of file to read
    
  '''

  row_num = 0
  isAccepted = 0

  #get size of file
  size = os.path.getsize(filename)

  #get number of lines in file
  with open(filename, "r+") as f:
    NUM_LINES = sum(1 for l in f)
    #truncate file to size
    f.truncate(size)

  with open(filename, "r") as f:
    trials = []
    #skip to accepted parameter
    for x in range (0, 11):
      line = f.readline()
    row_num += 11
    line = ''.join(line.split())
    #read num of accepted params
    accepts = int(line[10:12])

    #skip to accept parameter
    for x in range(11, 23):
      line = f.readline()
    row_num += 12
    line = ''.join(line.split())
    #get number value that corresponds to if data is accepted or not
    isAccepted = int(line[8])
  
    #loop through file
    while line:
      #skip to data
      for x in range(0, 5):
        line = f.readline()
      row_num += 5
      #if accepted = 1, read data lines
      if isAccepted == 1:
        #read 1101 rows of data
        sample = np.zeros(shape=(1101,64))
        sample_row = 0
        line_num = row_num
        for x in range(0, 1101):
          #turn line of data into numpy array
          line_array = np.fromstring(line.strip(), dtype=float, sep="\t")
          #delete non-EEG channels   
          line_array = np.delete(line_array, [64, 65])
          #print ("line #: ", line_num, " array: ", line_array) #debugging
          #add data to sample
          sample[sample_row] = line_array
          sample_row += 1
          #read next line
          line = f.readline()
          line_num+=1
        #add trial to list
        trials.append(sample)
      #else, skip to next accept parameter
      else:
        for x in range(0, 1101):
          line = f.readline()
      row_num += 1101
      #get next accept parameter (if not at end of file)
      if row_num < NUM_LINES:
        for x in range(0, 2):
          line = f.readline()
        row_num += 2
        line = ''.join(line.split())
        isAccepted = int(line[8])

  return np.asarray(trials)

**Static files**

In [ ]:
#list to hold samples
static_samples = []

labels = []
groups = []

#read files and add them to samples array
for x in range (0, len(static_files)):
  sample = read_file(static_files[x])
  #set labels and groups for trials appropriately
  trials = len(sample)
  labels.extend(static_labels[x] for y in range(0, trials))
  groups.extend(static_groups[x] for y in range(0, trials))
  print (static_files[x], " shape: ", sample.shape)
  static_samples.append(sample)

#update labels and groups
static_labels = np.asarray(labels)
static_groups = np.asarray(groups)

In [ ]:
#concatenate 3d numpy arrays to make one 3d numpy array
static_trials = static_samples[0]
for x in range (1, len(static_samples)):
  static_trials = np.concatenate((static_trials, static_samples[x]), axis=0)

print ("Static trials: ", static_trials.shape) 

**Dynamic files**

In [ ]:
dyn_samples = []

labels = []
groups = []

#read files and add them to array
for x in range(0, len(dyn_files)):
  sample = read_file(dyn_files[x])
  #set labels and groups for each trial accordingly
  trials = len(sample)
  labels.extend(dyn_labels[x] for y in range(0, trials))
  groups.extend(dyn_groups[x] for y in range(0, trials))
  print (dyn_files[x], " shape: ", sample.shape)
  dyn_samples.append(sample)

#update labels and groups
dyn_labels = np.asarray(labels)
dyn_groups = np.asarray(groups)

In [ ]:
#concatenate 3d numpy arrays to one 3d numpy array
dyn_trials = dyn_samples[0]
for x in range (1, len(dyn_samples)):
  dyn_trials = np.concatenate((dyn_trials, dyn_samples[x]), axis=0)

print ("Dynamic trials: ", dyn_trials.shape) 

**Fear files for when needed**

In [ ]:
'''uncomment when needed
fear_samples = []

labels = []
groups = []

#read files and add them to samples array
for x in range (0, len(fear_files)):
  sample = read_file(fear_files[x])
  #set labels and groups for trials appropriately
  trials = len(sample)
  labels.extend(fear_labels[x] for y in range(0, trials))
  groups.extend(fear_groups[x] for y in range(0, trials))
  print (fear_files[x], " shape: ", sample.shape)
  fear_samples.append(sample)

#update labels and groups
fear_labels = np.asarray(labels)
fear_groups = np.asarray(groups)
'''

In [ ]:
'''uncomment when needed
#concatenate 3d numpy arrays to one 3d numpy array
fear_trials = fear_samples[0]
for x in range (1, len(fear_samples)):
  fear_trials = np.concatenate((fear_trials, fear_samples[x]), axis=0)

print ("Fear trials: ", fear_trials.shape) 
'''

**Anger files for when needed**

In [ ]:
'''uncomment when needed
anger_samples = []

labels = []
groups = []

#read files and add them to samples array
for x in range (0, len(anger_files)):
  sample = read_file(anger_files[x])
  #set labels and groups for trials appropriately
  trials = len(sample)
  labels.extend(anger_labels[x] for y in range(0, trials))
  groups.extend(anger_groups[x] for y in range(0, trials))
  print (anger_files[x], " shape: ", sample.shape)
  anger_samples.append(sample)

#update labels and groups
anger_labels = np.asarray(labels)
anger_groups = np.asarray(groups)
'''

In [ ]:
'''uncomment when needed
#concatenate 3d numpy arrays to one 3d numpy array
anger_trials = anger_samples[0]
for x in range (1, len(anger_samples)):
  anger_trials = np.concatenate((anger_trials, anger_samples[x]), axis=0)

print ("Anger trials: ", anger_trials.shape) 
'''